In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GroceryBot, a sample grocery and recipe assistant - RAG + ReAct

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/chatbots/grocerybot_assistant.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/chatbots/grocerybot_assistant.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/chatbots/grocerybot_assistant.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


## 개요
이 노트북은 RAG(검색 증강 생성) 및 ReAct(추론 + 행동)를 사용하여 식료품 쇼핑 여정에서 고객을 지원할 수 있는 대화형 봇을 만드는 방법을 보여줍니다.

이 두 가지 접근 방식에 대한 자세한 내용을 알아보려면 관련 논문인 [RAG Arxiv 논문](https://arxiv.org/pdf/2005.11401.pdf) 및 [ReAct Arxiv 논문](https://arxiv. org/abs/2210.03629.pdf)

## 시나리오
당신이 가장 좋아하는 식료품점인 Cymbal Grocery의 사용자라고 상상해 보십시오. 라자냐처럼 저녁 식사로 좋은 음식을 요리하고 싶지만 어디서부터 시작해야 할지, 어떤 재료를 사야 할지, 라자냐를 요리하는 방법을 모릅니다.

웹사이트에 들어가니 Cymbal Grocery가 방금 새로운 대화형 봇인 GroceryBot을 출시한 것을 발견했습니다!

GroceryBot은 다음을 통해 쇼핑 여정에 도움을 드립니다.

1. 레시피 제안
2. 재료 목록 및 요리 지침 얻기
3. 해당 레시피에 맞춰 사고 싶은 제품 추천
4. 저녁 식사로 사고 싶은 새로운 제품을 찾을 수 있도록 도와드립니다!

## 목표 및 요구 사항
귀하의 목표는 **GroceryBot**을 개발하는 것입니다!

한 가지 주요 요구 사항이 있습니다. 이 봇이 **grounded**되어 있는지 확인해야 합니다. Grounding는 LLM을 데이터베이스와 같은 외부 지식 소스와 연결하는 프로세스를 의미합니다.

실제로 이는 GroceryBot이 다음을 활용해야 함을 의미합니다.

1. Cymbal Grocery의 기존 레시피 카탈로그입니다. GroceryBot은 이 카탈로그에 포함되지 않은 레시피를 제안해서는 안 됩니다.
2. Cymbal Grocery의 기존 제품 카탈로그입니다. GroceryBot은 이 카탈로그에 포함되지 않은 제품을 제안해서는 안 됩니다.
3. 레시피에 제안된 미리 계산된 제품 세트입니다.

이를 위해 LLM으로 전송되는 프롬프트에 사실 정보(이 경우 레시피 및 제품 정보)를 삽입하여 hallucination 문제를 완화하려고 시도하는 RAG(검색 증강 생성)라는 접근 방식을 사용할 수 있습니다.

The following image shows what could be possible with GroceryBot if the solution was to be deployed and integrated with a FrontEnd application.

![image](https://storage.googleapis.com/github-repo/img/language/reference_architectures/spotbot/spotbot_chat_example.png)


### GroceryBot 구현


이 시스템은 Vertex AI Generative 모델과 LangChain을 기반으로 합니다. LangChain을 처음 사용하는 경우 [이 노트북](https://github.com/GoogleCloudPlatform/generative-ai/blob/dev/언어/examples/langchain/intro_langchain_palm_api.ipynb)을 통해 익숙해지는 것이 좋습니다.

앞서 언급했듯이 모델을 기초로 삼으려면 LLM을 회사 내부 데이터베이스에 연결해야 합니다. 의사 결정을 내리고 언제 할지 결정할 수 있는 [ReAct like](https://ai.googleblog.com/2022/11/react-synergizing-reasoning-and-acting.html) 에이전트를 LangChain에 구현하면 됩니다. 이 데이터베이스를 쿼리해 보세요. LangChain의 에이전트에 대해 더 자세히 알고 싶으시면 [이 페이지](https://python.langchain.com/en/latest/modules/agents.html)를 방문하세요.

데모 목적으로 이 노트북은 로컬 데이터베이스만 사용합니다. 다음 설정이 채택됩니다.
- 제품 및 레시피 카탈로그는 [Faiss](https://python.langchain.com/en/latest/modules/indexes/Vectorstores/examples/faiss.html)를 사용하여 로컬로 정의됩니다. 몇 가지 예시 이상으로 확장해야 하는 프로덕션 시나리오에서는 [Vertex AI Matching Engine](https://cloud.google.com/vertex-ai/docs/matching-engine/overview) 관리형 벡터를 살펴보는 것이 좋습니다. [ScaNN 유사성 검색](https://ai.googleblog.com/2020/07/announce-scann-efficient-Vector.html)을 활용하는 Vertex AI의 데이터베이스 부분입니다.
- 레시피의 세부정보와 해당 레시피에 맞는 구매 추천 제품도 로컬에 저장됩니다. 프로덕션 시나리오에서는 [Cloud Datastore](https://cloud.google.com/datastore)와 같은 NoSQL 데이터베이스를 사용하여 이를 저장할 수 있습니다.

에이전트의 다양한 구성 요소와 데이터베이스와의 예상되는 상호 작용을 보여주는 다이어그램을 아래에서 볼 수 있습니다.

![image](https://storage.googleapis.com/github-repo/img/language/reference_architectures/spotbot/spotbot_architecture.png)

### Costs

This tutorial uses billable components of Google Cloud:
- Vertex AI Generative AI Studio

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

# Getting Started

### Install libraries

In [ ]:
!pip install --upgrade google-cloud-aiplatform==1.35.0 langchain==0.0.323 faiss-cpu==1.7.4 --user

***Colab only***: Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

### Import libraries

**Colab only:** Uncomment the following cell to initialize the Vertex AI SDK. For Vertex AI Workbench, you don't need to run this.

In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import glob
import pprint
from typing import Any, Iterator, List

from langchain.agents import AgentType, initialize_agent
from langchain.document_loaders import TextLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document
from langchain.tools import tool
from langchain.vectorstores import FAISS
from langchain.vectorstores.base import VectorStoreRetriever
from tqdm import tqdm

### Initialize models

In [ ]:
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0,
    top_p=0.8,
    top_k=40,
)

embedding = VertexAIEmbeddings()

# Create the Recipe & Product retrievers

앞서 언급했듯이 목표는 LLM에 더 많은 컨텍스트를 제공하기 위해 폐쇄 도메인 데이터베이스의 정보를 활용하는 것입니다. 이를 위해 LangChain에서 두 개의 로컬 벡터 데이터베이스와 상호 작용할 수 있는 두 개의 검색기를 생성합니다. 하나는 제품 항목용이고 다른 하나는 레시피 항목용입니다.

**일회성 프로세스**로 모든 제품과 레시피 항목이 임베딩으로 변환되어 관련 벡터 데이터베이스로 수집됩니다.

**검색 시** 쿼리(예: 라자냐)는 임베딩으로 변환되고, 쿼리에 가장 가까운 항목(예: lasagne al forno, 채식주의자 라자냐)을 찾기 위해 벡터 유사성 검색이 수행됩니다.

이 두 데이터베이스를 강화하기 위해 [Vertex AI Generative AI 모델](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#)을 사용하여 생성된 더미 레시피 및 제품 세트를 사용합니다. 기초_모델).

VectorStore에서 이 데이터를 로드하려면 [LangChain TextLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.text.TextLoader.html)를 사용하게 됩니다.

온라인에 공개된 기존 레시피 및 제품 세트로 이 접근 방식을 테스트하려면 [LangChain WebBaseLoader](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples)를 사용할 수 있습니다. /web_base.html).

First, fetch the dummy data on products and recipes from a public Cloud Storage bucket and store it locally.

In [ ]:
!gsutil -m cp -r "gs://github-repo/use-cases/grocery_bot/*" .

You then define a set of functions to enable the creation of the two vector databases, one for products, and one for recipes.

In [ ]:
def chunks(lst: List[Any], n: int) -> Iterator[List[Any]]:
    """Yield successive n-sized chunks from lst.

    Args:
        lst: The list to be chunked.
        n: The size of each chunk.

    Yields:
        A list of the next n elements from lst.
    """

    for i in range(0, len(lst), n):
        yield lst[i : i + n]


def load_docs_from_directory(dir_path: str) -> List[Document]:
    """Loads a series of docs from a directory.

    Args:
      dir_path: The path to the directory containing the docs.

    Returns:
      A list of the docs in the directory.
    """

    docs = []
    for file_path in glob.glob(dir_path):
        loader = TextLoader(file_path)
        docs = docs + loader.load()
    return docs


def create_retriever(top_k_results: int, dir_path: str) -> VectorStoreRetriever:
    """Create a recipe retriever from a list of top results and a list of web pages.

    Args:
        top_k_results: number of results to return when retrieving
        dir_path: List of web pages.

    Returns:
        A recipe retriever.
    """

    BATCH_SIZE_EMBEDDINGS = 5
    docs = load_docs_from_directory(dir_path=dir_path)
    doc_chunk = chunks(docs, BATCH_SIZE_EMBEDDINGS)
    for index, chunk in tqdm(enumerate(doc_chunk)):
        if index == 0:
            db = FAISS.from_documents(chunk, embedding)
        else:
            db.add_documents(chunk)

    retriever = db.as_retriever(search_kwargs={"k": top_k_results})
    return retriever

이제 이전 단계에서 정의한 함수를 사용하여 벡터 DB를 생성할 준비가 되었습니다.
각 Vector DB는 쿼리가 제공되면 해당 쿼리와 일치하는 문서 목록을 제공하는 Python 개체인 검색기 인스턴스를 제공합니다.

다음을 생성합니다:
- `recipe_retriever`: 쿼리와 일치하는 레시피 세트를 검색합니다.
- `product_retriever`: 쿼리와 일치하는 제품 세트를 검색합니다.

In [ ]:
recipe_retriever = create_retriever(top_k_results=2, dir_path="./recipes/*")
product_retriever = create_retriever(top_k_results=5, dir_path="./products/*")

이제 리트리버를 테스트할 준비가 되었습니다! 예를 들어 `recipe_retriever`에 '라자냐 요리법'을 찾으라고 요청하면 쿼리와 일치하는 가장 가까운 요리법 2개가 표시되어야 합니다.

In [ ]:
docs = recipe_retriever.get_relevant_documents("Any lasagne recipes?")
pprint.pprint([doc.metadata for doc in docs])

사용자가 토마토를 쿼리하는 경우 product_retriever와 유사한 동작을 얻게 됩니다.

In [ ]:
docs = product_retriever.get_relevant_documents("Any Tomatoes?")
pprint.pprint([doc.metadata for doc in docs])

`recipe_retriever`는 2개의 문서만 반환하는 반면 `product_retriever`는 5개의 문서를 반환하는 방법에 유의하세요. `create_retriever` 함수의 `top_k_results` 매개변수를 변경하여 모든 검색기가 반환하는 문서의 양을 변경할 수 있습니다.

## Agent

이제 검색기를 만들었으므로 ReAct와 유사한 접근 방식을 구현할 LangChain 에이전트를 만들 차례입니다.

에이전트는 잠재적으로 모든 기능을 수행할 수 있는 Python 함수로 생각할 수 있는 도구 모음에 액세스할 수 있습니다. 에이전트 설정을 독특하게 만드는 것은 사용자 입력에 따라 호출할 도구와 호출 순서를 **자율적으로** 결정하는 기능입니다.

## 1. Agent tools

가장 먼저 만들어야 할 것은 에이전트가 사용할 도구입니다. 각 도구는 에이전트가 작업을 수행하는 데 사용되므로 도구의 기능을 잘 설명하는 것이 중요합니다.

도구를 생성하는 방법에는 여러 가지가 있습니다. 자세한 내용은 [이 문서](https://python.langchain.com/docs/modules/agents/tools/how_to/custom_tools)를 참조하세요. 이 노트북은 `tool` 데코레이터 접근 방식을 사용합니다.

일부 도구에는 데코레이터에 `return_direct=True` 매개변수가 설정되어 있음을 알 수 있습니다. 이렇게 하면 도구의 출력이 LLM에 의해 사후 처리되지 않고 사용자에게 직접 반환됩니다.


먼저 이전에 정의한 두 개의 검색기 개체 `recipe_retriever` 및 `product_retriever`를 활용하는 두 개의 도구를 만듭니다.

In [ ]:
@tool(return_direct=True)
def retrieve_recipes(query: str) -> str:
    """
    Searches the recipe catalog to find recipes for the query.
    Return the output without processing further.
    """
    docs = recipe_retriever.get_relevant_documents(query)

    return (
        f"Select the recipe you would like to explore further about {query}: [START CALLBACK FRONTEND] "
        + str([doc.metadata for doc in docs])
        + " [END CALLBACK FRONTEND]"
    )

In [ ]:
@tool(return_direct=True)
def retrieve_products(query: str) -> str:
    """Searches the product catalog to find products for the query.
    Use it when the user asks for the products available for a specific item. For example `Can you show me which onions I can buy?`
    """
    docs = product_retriever.get_relevant_documents(query)
    return (
        f"I found these products about {query}:  [START CALLBACK FRONTEND] "
        + str([doc.metadata for doc in docs])
        + " [END CALLBACK FRONTEND]"
    )

레시피를 선택하는 사용자의 작업을 캡처하기 위해 에이전트가 사용하는 도구인 `recipe_selector`를 정의합니다. 레시피의 경로는 해당 레시피의 식별자로 사용됩니다.

In [ ]:
@tool
def recipe_selector(path: str) -> str:
    """
    Use this when the user selects a recipe.
    You will need to respond to the user telling what are the options once a recipe is selected.
    You can explain what are the ingredients of the recipe, show you the cooking instructions or suggest you which products to buy from the catalog!
    """
    return "Great choice! I can explain what are the ingredients of the recipe, show you the cooking instructions or suggest you which products to buy from the catalog!"

네 번째 도구를 사용하면 에이전트는 레시피 경로를 통해 레시피의 세부정보를 찾을 수 있습니다. 주어진 레시피에 대한 재료와 지침을 관찰하여 반환합니다. 그런 다음 에이전트는 이 정보를 사용하여 사용자의 특정 쿼리에 응답합니다.

In [ ]:
docs = load_docs_from_directory("./recipes/*")
recipes_detail = {doc.metadata["source"]: doc.page_content for doc in docs}


@tool
def get_recipe_detail(path: str) -> str:
    """
    Use it to find more information for a specific recipe, such as the ingredients or the cooking steps.
    Use this to find what are the ingredients for a recipe or the cooking steps.

    Example output:
    Ingredients:

    * 1 pound lasagna noodles
    * 1 pound ground beef
    * 1/2 cup chopped onion
    * 2 cloves garlic, minced
    * 2 (28 ounce) cans crushed tomatoes
    * 1 (15 ounce) can tomato sauce
    * 1 teaspoon dried oregano

    Would you like me to show you the suggested products from the catalogue?
    """
    try:
        return recipes_detail[path]
    except KeyError:
        return "Could not find the details for this recipe"

마지막으로 에이전트가 특정 레시피에 가장 적합한 제품을 찾을 수 있는 도구를 정의합니다. 데모 목적으로 이 정보는 사전에 하드코드되어 있습니다.

In [ ]:
@tool(return_direct=True)
def get_suggested_products_for_recipe(recipe_path: str) -> str:
    """Use this only if the user would like to buy certain products connected to a specific recipe example 'Can you give me the products I can buy for the lasagne?'",

    Args:
        recipe_path: The recipe path.

    Returns:
        A list of products the user might want to buy.
    """
    recipe_to_product_mapping = {
        "./recipes/lasagne.txt": [
            "./products/angus_beef_lean_mince.txt",
            "./products/large_onions.txt",
            "./products/classic_carrots.txt",
            "./products/classic_tomatoes.txt",
        ]
    }

    return (
        "These are some suggested ingredients for your recipe [START CALLBACK FRONTEND] "
        + str(recipe_to_product_mapping[recipe_path])
        + " [END CALLBACK FRONTEND]"
    )

## Creating the agent

이제 도구를 정의했으므로 에이전트를 생성할 준비가 되었습니다. 대화를 할 수 있도록 상담원에게 추억을 제공합니다.

에이전트는 'CONVERSATIONAL_REACT_DESCRIPTION' 유형으로 초기화됩니다. 이에 대해 자세히 알아보려면 [상대 문서](https://python.langchain.com/docs/modules/agents/agent_types/chat_conversation_agent) 및 [기타 에이전트 유형](https://python.langchain)을 살펴보세요. .com/docs/modules/agents/agent_types/).

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")
memory.clear()

tools = [
    retrieve_recipes,
    retrieve_products,
    get_recipe_detail,
    get_suggested_products_for_recipe,
    recipe_selector,
]
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
)

### Let's cook lasagne!

In [ ]:
agent.run("I would like to cook some lasagne. What are the recipes available?")

In [ ]:
agent.run("Selecting ./recipes/lasagne.txt")

In [ ]:
agent.run("Yes, can you give me the ingredients for that recipe?")

In [ ]:
agent.run("Can you give me the cooking instructions for that recipe?")

In [ ]:
agent.run("Can you give me the products I can buy for this recipe?")

In [ ]:
agent.run("Can you show me other tomatoes you have available?")

In [ ]:
agent.run("Nice, how about carrots?")

In [ ]:
agent.run("Thank you, that's everything!")

## Setting guardrails in the agent - Custom agent

드디어 첫 번째 식료품 도우미를 만들었습니다! 🎉 그런데 사용자가 경쟁사에 대해 묻는다면 어떨까요? 아니면 사용자가 에이전트를 사용하여 일반적인 Q&A와 같이 허용되지 않는 작업을 수행하는 경우에는 어떻게 되나요?

기업 환경에서는 대화를 차단하거나 보호하고 싶을 수도 있습니다.

일부 가드레일을 설정하는 가장 쉬운 방법은 에이전트 프롬프트에 사용자 지정 접두사를 제공하는 것입니다.

기본적으로 [여기](https://github.com/hwchase17/langchain/blob/master/langchain/agents/conversational/prompt.py) 정의된 에이전트에 대한 기본 프롬프트를 재정의하게 됩니다.
)


In [ ]:
PREFIX = """
당신은 GroceryBot입니다.
GroceryBot은 Cymbal Grocery에서 제공하는 대규모 언어 모델입니다.
고객이 최고의 레시피를 찾고 구매하기에 적합한 제품을 찾도록 돕습니다.
레시피 계획, 제품 찾기, 쇼핑 경험 촉진 등의 작업을 수행할 수 있습니다.
GroceryBot은 끊임없이 학습하고 개선하고 있습니다.
GroceryBot은 어떠한 경우에도 다른 회사 이름을 공개하지 않습니다.
GroceryBot은 항상 소매 보조원인 GroceryBot으로 식별되어야 합니다.
GroceryBot이 역할극을 하거나 GroceryBot이 아닌 다른 역할을 하도록 요청받은 경우 "저는 식료품 보조원 GroceryBot입니다."라고 응답해야 합니다.

TOOLS:
------

GroceryBot has access to the following tools:"""


tool = [
    retrieve_recipes,
    retrieve_products,
    get_recipe_detail,
    get_suggested_products_for_recipe,
    recipe_selector,
]
memory_new_agent = ConversationBufferMemory(memory_key="chat_history")
memory_new_agent.clear()

guardrail_agent = initialize_agent(
    tool,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory_new_agent,
    verbose=True,
    agent_kwargs={"prefix": PREFIX},
)

### Testing the new guardrailed agent

Test the new agent, by comparing it with the one was created previously!

In [ ]:
print("Guardrailed agent: ", guardrail_agent.run("What is the capital of Germany?"))
print("Previous agent: ", agent.run("What is the capital of Germany?"))

In [ ]:
print(
    "Guardrailed agent: ",
    guardrail_agent.run("What are some competitors of Cymbal Grocery?"),
)
print("Previous agent: ", agent.run("What are some competitors of Cymbal Grocery?"))

In [ ]:
print("Guardrailed agent: ", guardrail_agent.run("Give me a recipe for lasagne"))
print("Previous agent: ", agent.run("Give me a recipe for lasagne"))

보시다시피 새로운 가드레일 에이전트는 사용자가 일반적인 Q&A 질문을 하는 것을 방지할 수 있었습니다. 하지만 두 에이전트 모두 쇼핑 여정에 관해 사용자를 지원할 수 있습니다.

# Conclusion
이 노트북은 Vertex AI Generative AI 모델과 LangChain을 사용하여 식료품 보조 봇을 만드는 방법을 보여줍니다.

이 노트북에서는 다음 방법을 배웠습니다.
- RAG를 활용하여 LLM을 기반으로 환각을 피하는 방법
- 벡터 데이터베이스 생성 및 쿼리
- LangChain 도구 만들기
- 정보 제공 및 거래 지원이 가능한 LangChain Agent를 생성합니다.
- 기업 환경에 대비할 수 있도록 에이전트를 보호합니다.